# see Danieles Mail from 28.01.25

In [1]:
from utils.binaries import *
from utils.plotting import *
from utils.auger.sd import randoms

17:34:58 (   +2.1s) [INFO   ] -- import logging
17:34:58 (  +168ms) [INFO   ] -- import numpy as np
17:34:58 (   +13ms) [INFO   ] -- import uncertainties
17:34:58 (  +421ms) [INFO   ] -- import pandas as pd
17:34:58 (    +2ms) [WARNING] -- pandas support to be dropped at some point!
17:34:58 (    +2ms) [INFO   ] -- import binaries.tools as tools
17:34:59 (  +412ms) [INFO   ] -- import matplotlib.pyplot as plt
17:34:59 (  +589ms) [INFO   ] -- import seaborn as so
17:34:59 (    +3ms) [INFO   ] -- set MONI_PATH = PosixPath('/cr/work/filip/monit_and_sd')
17:34:59 (    +1ms) [INFO   ] -- set HIST_PATH = PosixPath('/cr/work/filip/monit_and_sd')
17:34:59 (    +1ms) [INFO   ] -- set PLOT_PATH = PosixPath('/cr/data01/filip/plots')
17:34:59 (    +0ms) [INFO   ] -- set DATA_PATH = PosixPath('/cr/data01/filip/Data')
17:34:59 (    +1ms) [INFO   ] -- set SCAN_PATH = PosixPath('/cr/data01/filip/xy-calibration')
17:34:59 (    +1ms) [INFO   ] -- set OFLN_PATH = PosixPath('/cr/data01/filip/offline/insta

In [3]:
RandomFiles = randoms.UubRandom(station = "Svenja", detectors = "ssd")

total_time = 0
px = np.zeros(10000)
pulse_heights = []

for File in tools.ProgressBar(RandomFiles):
    for trace in File:

        total_time += 8.33e-9 * len(trace['trace'])
        _max = np.max(trace['trace'])
        px[_max] += 1

        # if _max >= 70:
        #     pulse_heights.append(_max)

tools.pickle_save('pulse_height_distribution.pkl', np.dstack([np.arange(len(px)) / trace['mip_peak'], px / total_time]))
# print(f"{np.mean(pulse_heights) = :.4f}")
# print(f"{np.std(pulse_heights) = :.4f}")

800/800 [********************] || 00:11:38>00:00:00,         1.15 it/s


In [ ]:
px = tools.pickle_load('pulse_height_distribution.pkl')

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

px[:70] = 0
cumsum = np.cumsum(px[::-1])[::-1]

ax1.plot(range(10000) / trace['mip_peak'], px/cumsum[0], label='pdf')
ax1.set_yscale('log')
ax1.set_xscale('log')
ax1.legend()

ax2.plot(range(10000) / trace['mip_peak'], cumsum / (cumsum[0] * 2048 * 8.33e-9), label='int. rate')
ax2.set_yscale('log')
ax2.legend()

In [ ]:
# fitting rate-threshold relationship

from scipy.optimize import curve_fit

def exponential(x, A, mu):
    return A * np.exp(x * mu)

def exponential_corrected(x, A, mu, c, d, e, f, g):
    return A * np.exp(x * mu) + np.poly1d([c, d, e, f, g])(x)

x, y = np.loadtxt('mean_rates_t3_mip.txt', unpack=True)
ERROR = np.sqrt(y / 60)
NDOF = len(x) - 7
bounds_low = [0, -np.inf] + [-np.inf] * 5
bounds_high = [np.inf, 0] + [np.inf] * 5

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(x, y, marker='none')
popt, pcov = curve_fit(exponential, x, y,
                       sigma=ERROR,
                       bounds=(bounds_low[:2], bounds_high[:2]))
ax1.plot(x, exponential(x, *popt), marker='none')

popt, pcov = curve_fit(exponential_corrected, x, y,
                       sigma=ERROR,
                       bounds=(bounds_low, bounds_high))


ax1.plot(x, exponential_corrected(x, *popt), marker='none')
ax1.set_ylim(0.01, 240)
ax1.set_yscale('log')

ax2.plot(x, (y - exponential_corrected(x, *popt)) / ERROR)
ax2.legend(title=fr"$\chi_\nu^2$ = {np.sum((y - exponential_corrected(x, *popt))**2 / ERROR**2 / NDOF):.2f}")
# ax2.set_yscale('log')

In [ ]:
X_TARGET = 5
Y_TARGET = 10

print(X_TARGET, y[np.argmin(np.abs(x-X_TARGET))])
print(x[np.argmin(np.abs(y-Y_TARGET))], Y_TARGET)

In [ ]:
def pdf(x, a, mu):
    return a * np.exp(x*mu)

popt_pdf, pcov = curve_fit(pdf, np.arange(70,1000)/48.94, px[70:1000]/cumsum[0],
                       bounds=([0, -np.inf],[np.inf, 0]))

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

X = np.arange(70, 1000) / trace['mip_peak']
ax1.plot(X, pdf(X, *popt_pdf))
ax1.plot(range(1000) / trace['mip_peak'], px[:1000]/cumsum[0], marker='none')

ax2.plot(range(70,1000) / trace['mip_peak'], px[70:1000]/cumsum[0] - pdf(X, *popt_pdf), marker='none')

In [ ]:
x  = range(10000) / trace['mip_peak']
px = popt_pdf #px/cumsum[0]
fx = popt #lambda x: exponential_corrected(x, *popt)

tools.pickle_save('x_px_fx.pkl', [x, px, fx])

In [ ]:
trace['mip_peak']

In [ ]:
plt.plot(range(len(px)), px / (cumsum[0] * 2048 * 8.33e-9))
plt.yscale('log')
plt.xlim(0, 1000)

In [ ]:
trace['mip_peak']